<a href="https://colab.research.google.com/github/syedayazsa/Realtime-Face-Indentification/blob/master/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [1]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from PIL import Image
import base64
from glob import glob
import random
import cv2
import os
from keras.models import load_model
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.preprocessing import image
import pandas as pd
import multiprocessing as mp
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19OiBwc-NrJVa8o52GkXcmxrGfYiOLCtx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19OiBwc-NrJVa8o52GkXcmxrGfYiOLCtx" -O facedetection_vgg.h5 && rm -rf /tmp/cookies.txt

--2020-05-28 07:13:15--  https://docs.google.com/uc?export=download&confirm=dBIO&id=19OiBwc-NrJVa8o52GkXcmxrGfYiOLCtx
Resolving docs.google.com (docs.google.com)... 172.217.13.78, 2607:f8b0:4004:808::200e
Connecting to docs.google.com (docs.google.com)|172.217.13.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ac-docs.googleusercontent.com/docs/securesc/ptq38q271q12thrl8ar691ebnscev25j/43poph5lqh3vslcr0tl82oetp94nspqg/1590649950000/14211000813324722452/18009871134738132482Z/19OiBwc-NrJVa8o52GkXcmxrGfYiOLCtx?e=download [following]
--2020-05-28 07:13:15--  https://doc-08-ac-docs.googleusercontent.com/docs/securesc/ptq38q271q12thrl8ar691ebnscev25j/43poph5lqh3vslcr0tl82oetp94nspqg/1590649950000/14211000813324722452/18009871134738132482Z/19OiBwc-NrJVa8o52GkXcmxrGfYiOLCtx?e=download
Resolving doc-08-ac-docs.googleusercontent.com (doc-08-ac-docs.googleusercontent.com)... 172.217.12.225, 2607:f8b0:4004:807::2001
Connecting to doc-08-

In [2]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lHtSjvVWP_VUVRjmMBVXQa_9WR2AnB3r' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1lHtSjvVWP_VUVRjmMBVXQa_9WR2AnB3r" -O haarcascade_frontalface_default.xml && rm -rf /tmp/cookies.txt

--2020-05-28 07:13:23--  https://docs.google.com/uc?export=download&confirm=&id=1lHtSjvVWP_VUVRjmMBVXQa_9WR2AnB3r
Resolving docs.google.com (docs.google.com)... 142.250.31.113, 142.250.31.102, 142.250.31.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.31.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-60-docs.googleusercontent.com/docs/securesc/jbju83ubigc3qdrcaq5ui6kaiebhv7j4/arhqtjd2unv3v0hhlutlo23l5i5vcqof/1590649950000/14211000813324722452/02065955666585418514Z/1lHtSjvVWP_VUVRjmMBVXQa_9WR2AnB3r?e=download [following]
--2020-05-28 07:13:23--  https://doc-0c-60-docs.googleusercontent.com/docs/securesc/jbju83ubigc3qdrcaq5ui6kaiebhv7j4/arhqtjd2unv3v0hhlutlo23l5i5vcqof/1590649950000/14211000813324722452/02065955666585418514Z/1lHtSjvVWP_VUVRjmMBVXQa_9WR2AnB3r?e=download
Resolving doc-0c-60-docs.googleusercontent.com (doc-0c-60-docs.googleusercontent.com)... 172.217.12.225, 2607:f8b0:4004:807::2001
Connecting t

## Loading the cascade and model

In [0]:
model = load_model('/content/facedetection_vgg.h5')
face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')

## Function to extract face

In [0]:
def face_extractor(img):
    # Function detects faces and returns the cropped face
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

## Predict and classify functions

In [0]:
def predict(p):
    array = model.predict(p)
    result = array[0]
    answer = np.argmax(result)
    return answer

In [0]:
def classify(c):
    if c==0:
        c_img="person0"
    elif c==1:
        c_img="person1"
    elif c==2:
        c_img="person2"
    elif c==3:
        c_img="person3"
    elif c==4:
        c_img="person4"
    return c_img

# Face Recognition in local video file

## Creating a text file

In [0]:
f= open("data.txt","w+")

In [0]:
#adding columns to the text file
f= open("data.txt","w")
f.write('VideoName,PersonName,FrameNo\n')
f.close()

In [8]:
f= open("data.txt","r")
for lines in f:
  print(lines)

VideoName,PersonName,FrameNo



In [9]:
files = glob("/content/test/*")
print(files)

['/content/test/6822826321659440475.mp4', '/content/test/cam1-6828020019452055937.mp4']


## Counting total number of frames in the videos

In [10]:
cap = cv2.VideoCapture(files[0])
length_1 = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(length_1)

463


In [11]:
cap = cv2.VideoCapture(files[1])
length_2 = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(length_2)

471


# **Frame by frame prediction**
(Test video has been manually uploaded to colab file)

In [12]:
f = open('data.txt', 'a+')
file = files[0]
vid_name = os.path.basename(file)
video_capture = cv2.VideoCapture(file)
while True:
    _, frame = video_capture.read()
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because we trained the model with this image size.
        img_array = np.array(im)
        #Our keras model used a 4D tensor, (images x height x width x channel)
        #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        ans = predict(img_array)
        name = classify(ans)
        cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        if name is not None:
            f.write(str(vid_name)+','+name+','+str(video_capture.get(cv2.CAP_PROP_POS_FRAMES))+'\n')
            print(video_capture.get(cv2.CAP_PROP_POS_FRAMES))
        else:
          pass
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    #cv2.imshow('Video', frame)
    if video_capture.get(cv2.CAP_PROP_POS_FRAMES) == 463:
        break
video_capture.release()
cv2.destroyAllWindows()
f.close()

2.0
4.0
5.0
8.0
9.0
10.0
11.0
14.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
65.0
67.0
68.0
76.0
77.0
78.0
79.0
82.0
83.0
84.0
85.0
99.0
137.0
138.0
141.0
142.0
152.0
153.0
167.0
253.0
262.0
263.0
265.0
268.0
273.0
275.0
280.0
282.0
283.0
284.0
285.0
298.0
336.0
337.0
338.0
340.0
350.0
404.0


In [13]:
f = open('data.txt', 'a+')
file = files[1]
vid_name = os.path.basename(file)
video_capture = cv2.VideoCapture(file)
while True:
    _, frame = video_capture.read()
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because we trained the model with this image size.
        img_array = np.array(im)
        #Our keras model used a 4D tensor, (images x height x width x channel)
        #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        ans = predict(img_array)
        name = classify(ans)
        cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        if name is not None:
            f.write(str(vid_name)+','+name+','+str(video_capture.get(cv2.CAP_PROP_POS_FRAMES))+'\n')
            print(video_capture.get(cv2.CAP_PROP_POS_FRAMES))
        else:
          pass
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    #cv2.imshow('Video', frame)
    if video_capture.get(cv2.CAP_PROP_POS_FRAMES) == 463:
        break
video_capture.release()
cv2.destroyAllWindows()
f.close()

13.0
18.0
32.0
45.0
46.0
47.0
66.0
133.0
159.0
218.0
242.0
280.0
365.0
379.0
380.0
381.0
423.0
427.0
429.0
430.0
433.0
434.0
435.0


In [14]:
f= open("data.txt","r")
for lines in f:
  print(lines)

VideoName,PersonName,FrameNo

6822826321659440475.mp4,person0,2.0

6822826321659440475.mp4,person2,4.0

6822826321659440475.mp4,person2,5.0

6822826321659440475.mp4,person2,8.0

6822826321659440475.mp4,person0,9.0

6822826321659440475.mp4,person0,10.0

6822826321659440475.mp4,person0,11.0

6822826321659440475.mp4,person0,14.0

6822826321659440475.mp4,person4,50.0

6822826321659440475.mp4,person4,51.0

6822826321659440475.mp4,person1,52.0

6822826321659440475.mp4,person1,53.0

6822826321659440475.mp4,person2,54.0

6822826321659440475.mp4,person2,55.0

6822826321659440475.mp4,person4,56.0

6822826321659440475.mp4,person2,57.0

6822826321659440475.mp4,person2,58.0

6822826321659440475.mp4,person2,59.0

6822826321659440475.mp4,person2,60.0

6822826321659440475.mp4,person2,61.0

6822826321659440475.mp4,person2,62.0

6822826321659440475.mp4,person2,63.0

6822826321659440475.mp4,person1,65.0

6822826321659440475.mp4,person2,67.0

6822826321659440475.mp4,person2,68.0

6822826321659440475.mp4,p

## Reading Text as a dataframe and saving as csv file 

In [15]:
df = pd.read_csv('/content/data.txt', sep = ',')
df.head(10)

,VideoName,PersonName,FrameNo
0,6822826321659440475.mp4,person0,2.0
1,6822826321659440475.mp4,person2,4.0
2,6822826321659440475.mp4,person2,5.0
3,6822826321659440475.mp4,person2,8.0
4,6822826321659440475.mp4,person0,9.0
5,6822826321659440475.mp4,person0,10.0
6,6822826321659440475.mp4,person0,11.0
7,6822826321659440475.mp4,person0,14.0
8,6822826321659440475.mp4,person4,50.0
9,6822826321659440475.mp4,person4,51.0


In [16]:
df.shape

(83, 3)

In [0]:
df.to_csv('data.csv', encoding = 'utf-8')
from google.colab import files
files.download('data.csv')